**Read me**
1. Insert this file into Google Drive
2. Create a folder named 'videos' in Drive and upload your videos in it
3. Right click on 'pose_save_load.ipynb' and then click on open with Google Colaboratory
4. Run cells below

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 1.3 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pandas as pd
import openpyxl
import os
#mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic



After running cell below you may see some windows asking for your permission to connect to your Drive, so accept all and continue. 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This cell makes a list of files in 'videos' folder and creates a folder for saving excel files if not exists

In [6]:
video_folder_path = '/content/drive/MyDrive/videos' 
files = os.listdir(video_folder_path) 
excel_folder_path = '/content/drive/MyDrive/xls' 
if not os.path.exists(excel_folder_path): 
  os.makedirs(excel_folder_path)

This cell picks up each video file and extracts its pose landmarks and then saves the landmark coordinates and frame numbers in an '.xlsx' file in 'xls' folder.  

In [37]:
# Initiate holistic model
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
for item in files:
  cap = cv2.VideoCapture(video_folder_path + '/' + item)
  counter = 0
  failed = 0
  df = pd.DataFrame(columns= ['frame', 'x', 'y', 'z'])
  while cap.isOpened():
    ret, frame = cap.read()
    
    if ret == True:
      
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = holistic.process(image)
      if results.pose_landmarks:
        # saving to excel
        counter += 1
        data = [{'frame': counter, 'x': i.x,'y': i.y,'z': i.z} for i in results.pose_landmarks.landmark]
        df1 = pd.DataFrame(data = data)
        df = df.append(df1)
      else:
        failed += 1

      if cv2.waitKey(5) & 0xFF == 27:
        break

    else:
      break
  print('{} frames failed'.format(failed))
  name = excel_folder_path + '/' + item[:-3] + 'xlsx'
  df = df.astype(float)
  df.to_excel(name)

  cap.release()
  cv2.destroyAllWindows()

10 frames failed


This cell makes a list of excel files in 'xls' folder and creates a 'pose' folder to store the pose videos in '.avi' format.
From each '.xls' file it makes a video showing only landmarks and connections.

In [7]:
# Connections between points
connections = [i for i in mp_holistic.POSE_CONNECTIONS]
#print(connections)
excel_files = os.listdir(excel_folder_path)
res=(500,500) #resulotion
pose_folder_path = '/content/drive/MyDrive/pose'
if not os.path.exists(pose_folder_path):
  os.makedirs(pose_folder_path)

for item in excel_files:
  name = pose_folder_path + '/' + item[:-4] + 'avi'
  out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc(*'DIVX'), 60, res)

  df = pd.read_excel(excel_folder_path + '/' +item)

  counter = 1
  lmk = df.loc[(df['frame']==counter)]
  img_arr = []
  while not lmk.empty:
    points = [(int(lmk['x'][i]*res[0]),int(lmk['y'][i]*res[1])) for i in lmk.index]
    img = np.ones(shape=(res[0],res[1],3), dtype=np.uint8)

    for i,j in points:
      cv2.circle(img,(i,j), 3, (0,0,255), 0)

    for i,j in connections:
      cv2.line(img,points[i], points[j],(255,0,0), 1)

    img_arr.append(img) 
    counter += 1
    lmk = df.loc[(df['frame']==counter)]

  for i in range(len(img_arr)):
    out.write(img_arr[i])
    
  out.release()

